# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\аварии\Экс6.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.000000,0.111111,3,0.500000,0.200000,0.250000,0.111111
1,Экспертиза промышленной безопасности,9.000000,1.000000,8,1.000000,1.000000,5.000000,1.000000
2,Готовность к ликвидации последствий аварий и и...,0.333333,0.125000,1,0.333333,0.111111,0.142857,0.111111
3,Оформление информационными знаками и знаками б...,2.000000,1.000000,3,1.000000,1.000000,1.000000,0.333333
4,Содержание объектов и оборудования,5.000000,1.000000,9,1.000000,1.000000,7.000000,1.000000


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов        int64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0       1.000000                              0.111111   
1       9.000000                              1.000000   
2       0.333333                              0.125000   
3       2.000000                              1.000000   
4       5.000000                              1.000000   
5       4.000000                              0.200000   
6       9.000000                              1.000000   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                                  3         
1                                                  8         
2                                                  1         
3                                                  3         
4                                                  9         
5                                                  7         
6                                                  9         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов        int64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала       float64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,1.000000,0.111111,0.500000,0.200000,0.250000,0.111111
1,9.000000,1.000000,1.000000,1.000000,5.000000,1.000000
2,0.333333,0.125000,0.333333,0.111111,0.142857,0.111111
3,2.000000,1.000000,1.000000,1.000000,1.000000,0.333333
4,5.000000,1.000000,1.000000,1.000000,7.000000,1.000000
5,4.000000,0.200000,1.000000,0.142857,1.000000,0.166667
6,9.000000,1.000000,3.000000,1.000000,6.000000,1.000000


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                 30.333333
Экспертиза промышленной безопасности                          4.436111
Готовность к ликвидации последствий аварий и инцидентов      40.000000
Оформление информационными знаками и знаками безопасности     7.833333
Содержание объектов и оборудования                            4.453968
Охранные зоны (ОЗ)                                           20.392857
Организация рабочего процесса и безопасности персонала        3.722222
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.000000,0.111111,3,0.500000,0.200000,0.250000,0.111111,0.032967,0.025047,0.075,0.063830,0.044904,0.012259,0.029851
1,9.000000,1.000000,8,1.000000,1.000000,5.000000,1.000000,0.296703,0.225423,0.200,0.127660,0.224519,0.245184,0.268657
2,0.333333,0.125000,1,0.333333,0.111111,0.142857,0.111111,0.010989,0.028178,0.025,0.042553,0.024947,0.007005,0.029851
3,2.000000,1.000000,3,1.000000,1.000000,1.000000,0.333333,0.065934,0.225423,0.075,0.127660,0.224519,0.049037,0.089552
4,5.000000,1.000000,9,1.000000,1.000000,7.000000,1.000000,0.164835,0.225423,0.225,0.127660,0.224519,0.343257,0.268657


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.032967,0.025047,0.075,0.063830,0.044904,0.012259,0.029851
1,0.296703,0.225423,0.200,0.127660,0.224519,0.245184,0.268657
2,0.010989,0.028178,0.025,0.042553,0.024947,0.007005,0.029851
3,0.065934,0.225423,0.075,0.127660,0.224519,0.049037,0.089552
4,0.164835,0.225423,0.225,0.127660,0.224519,0.343257,0.268657
5,0.131868,0.045085,0.175,0.127660,0.032074,0.049037,0.044776
6,0.296703,0.225423,0.225,0.382979,0.224519,0.294221,0.268657


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.032967,0.025047,0.075,0.063830,0.044904,0.012259,0.029851,0.040551
1,0.296703,0.225423,0.200,0.127660,0.224519,0.245184,0.268657,0.226878
2,0.010989,0.028178,0.025,0.042553,0.024947,0.007005,0.029851,0.024075
3,0.065934,0.225423,0.075,0.127660,0.224519,0.049037,0.089552,0.122446
4,0.164835,0.225423,0.225,0.127660,0.224519,0.343257,0.268657,0.225621


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

0.296392639035743
1.8389307316899104
0.1746295552436848
0.9060812411837207
1.873800888900177
0.6629350756599204
2.1943979127928213


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

7.947168044505977


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.1578613407509962
1.4142857142857144
0.1116191298239367


In [20]:
# Получилось, что значение CR=0.11, что большее 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами